customer id : ID=5820,id: 2698

In [2]:
#for password
import sqlite3
connec = sqlite3.connect('abc.db')
cur = connec.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS password(pass text)''')
cur.execute('''INSERT INTO password VALUES('abc')''')
connec.commit()
connec.close()
        

In [1]:
#to delete the whole tables
import sqlite3
connec = sqlite3.connect('abc.db')
cur = connec.cursor()
cur.execute('''drop table movie_data2''')
cur.execute('''drop table customer_data2''')
cur.execute('''drop table password''')
connec.commit()
connec.close()


In [ ]:
#to show data of customer table
import sqlite3
connec = sqlite3.connect('abc.db')
cur = connec.cursor()

def ticket():
        cur.execute('''select * from customer_data2''')
        rows=cur.fetchall()
        for row in rows:
            print('Id:',row[0],', Moive:',row[1],', Timing:',row[2],', Hall:',row[3],', Total cost:',row[4],', Seats:',row[5])
        connec.commit()
        connec.close()
        
ticket()

In [9]:
#main program 
import sqlite3
import random
from IPython.display import clear_output
connec = sqlite3.connect('abc.db')
cur = connec.cursor()

class Hall:
    def add(self,hall):
        try:
            #this function addes new movies to a hall all movies should have a unique name and timing
            mov_name=input("\nenter movie name:").upper()
            hall=hall
            timing=input('enter the timing :')
            price=input("enter the ticket price :")
            seats=int(input("enter the number of seats :"))
            left_seats=seats
            last_seat=0
            cur.execute('''CREATE TABLE IF NOT EXISTS movie_data2(mov_name text  , hall text , timing text , price text ,seats integer,left_seats integer,last_seat integer,UNIQUE(mov_name,timing))''')
            cur.execute('''INSERT INTO movie_data2 VALUES(?,?,?,?,?,?,?)''',(mov_name,hall,timing,price,seats,left_seats,last_seat))
            connec.commit()
        except Exception as e:
            print("Error : ",e)

        

    def show(self):
        try:
            #display all shows available
            print("\n***************the details are *********************************\n")
            cur.execute('''select * from movie_data2;''')
            rows=cur.fetchall()
            for row in rows:
                print("  Movie : ,",row[0]," , Hall : ",row[1],", Timing : ",row[2],", Price : ",row[3],", Seats : ",row[4],", Remaing seats :",row[5]," last booked seat :",row[6])
            connec.commit()
        except Exception as e:
            print("Error : ",e)


    def replace(self):
        try:
            #replace an existing show with a new one
            self.show()
            movie=input("\nenter the moive which is to be replaced:").upper()
            new_mov_name=input("enter newmovie name :").upper()
            timing=input('enter the newtiming :')
            price=input("enter the new ticket price :")
            seats=int(input("enter the new number of seats :"))
            left_seats=seats
            last_seat=0
            statement="UPDATE movie_data2 SET mov_name = ? , timing = ? , price = ? , seats = ? ,left_seats=?,last_seat=? WHERE mov_name = ?"
            value=(new_mov_name,timing,price,seats,left_seats,last_seat,movie)
            cur.execute(statement,value)
            connec.commit()
            print('\n*********************Data Updated***********************')
        except Exception as e:
            print("Error : ",e)

    def book(self):
        try:
            #customer bookes a show 
            cus=Customer()
            id=random.randint(1000,9999)
            mov=input("\nenter movie name : ").upper()
            time=input("enter timeing:")
            seats=int(input("enter the number of seats :"))
            cur.execute('''select seats,left_seats,last_seat,price,hall from movie_data2 where mov_name=?''',(mov,))
            rows=cur.fetchall()
            noof_seats=rows[0][0]
            remaining_seats=rows[0][1]
            last_seat=rows[0][2]
            pri=rows[0][3]
            hall=rows[0][4]
            X=last_seat+1
            Y=last_seat+seats+1
            cost=int(pri)*seats
            #/////////////////
            remaining_seats=remaining_seats-seats
            if remaining_seats<=0:
                print("*******************Cannot book this much seats as less seats are available then U booked**************************")
            else:
                booked_seats=[i for i in range(X,Y)]
                last_seat=booked_seats[-1]
                print("**********Details**********")
                print("id:",id)
                print("movie:",mov)
                print("timing:",time)
                print("Hall:",hall)
                print("reserve seats:",booked_seats)
                print("price:",cost)
                print("**************................please save Ur id ...........***************")
                #////////////new customer database
                cur.execute('''CREATE TABLE IF NOT EXISTS customer_data2(id integer,mov_name text  , timing text , hall text , price text ,reserve_seats text ,UNIQUE(id))''')
                cur.execute('''INSERT INTO customer_data2 VALUES(?,?,?,?,?,?)''',(id,mov,time,hall,cost,str(booked_seats)))

                #//////////update movie db
                state="update movie_data2 set left_seatS=?,last_seat=? where mov_name=?"
                value=(remaining_seats,last_seat,mov)
                cur.execute(state,value)

                connec.commit()
        except Exception as e:
            print("Error : ",e)



#////////////////owner        

class Owner:
    def base(self):
        try:
            #base class of the owner
            a=Hall()
            select=int(input("\nenter \n 1->add new movie \n 2-> show all movies \n 3->change a Hall moive \n 4->change password\n"))
            if select==1:
                hall=input("enter the hall\n H1 \n H2 \n")
                if hall=='H1':
                    a.add(hall)
                elif hall=='H2':
                    a.add(hall)
                else:
                    print("invalid ")
            elif select==2:
                a.show()
            elif select==3:
                a.replace()
            elif select==4:
                self.Password()
            else:
                print("invalid")
        except Exception as e:
            print("Error : ",e)

    def Password(self):
        try:
            currpass=input("Current password:")
            new_pass=input("Enter new password:")
            con_pass=input("Confirm new password:")
            if con_pass==new_pass:
                state="update password set pass=? where pass=?"
                value=(new_pass,currpass)
                cur.execute(state,value)
                print("saved password")
            else:
                if new_pass!=con_pass:
                    print("Retry")
                else:
                    print("******MisMatch Password************")
                
        except Exception as e:
            print("Error : ",e)




#//////////////////////customer
class Customer:
    def base(self):
        #base class of customer
        try:
            option=int(input("option 1 : shows available \noption 2 : my ticket\n"))
            if option==1:
                self.shows()
            elif option==2:
                self.ticket()
            else:
                print("invalid option")
        except Exception as e:
            print("Error : ",e)


    def shows(self):
        #display shows availabe to customer
        try:
            a=Hall()
            print("The available shows are \n")
            cur.execute('''select mov_name,timing from movie_data2;''')
            rows=cur.fetchall()
            for row in rows:
                print("  Movie ",row[0],". Timing",row[1])
            connec.commit()

            option=int(input('\n****************Enter*******************\n 1 to Book Seats: \n 2 to return back:\n'))
            if option==1:
                a.book()
            elif option==2:
                self.base()
            else:
                print("invaid option")
        except Exception as e:
            print("Error : ",e)


    def ticket(self):
        #customer looks for his info by giving his id
        try:
            i=input("enter id:")
            cur.execute('''select * from customer_data2 where id = ?''',(i,))
            rows=cur.fetchall()
            if len(rows)==0:
                print("\n************No Data found***************\n")
            else:
                for row in rows:
                    print('Id:',row[0],', Moive:',row[1],', Timing:',row[2],', Hall:',row[3],', Total cost:',row[4],', Seats:',row[5])
            connec.commit()
        except Exception as e:
            print("Error : ",e)


#/////////////////MENU  
lst1=[]
def menu():
    try:
        select = int(input("Enter \n 1->owner \n 2->customer \n"))
        if select==1:
            if len(lst1)==0:
                password=input("enter password:")
            cur.execute('''select * from password;''')
            rows=cur.fetchall()
            for row in rows:
                #print(row)
                if len(lst1)!=0 or row[0]==password: 
                    print('**********Welcome***************')
                    owner=Owner()
                    owner.base()
                    if len(lst1)==0:
                        lst1.append(password)
                else:
                    print("*******Invalid password**********")
            connec.commit()
        elif select==2:
            customer=Customer()
            customer.base()
        else:
            print("invalid selection")
    except Exception as e:
        print("Error : ",e)
    
while True:
    menu()
    clear_output(wait=True)
    flag=int(input('\nto continie enter 1:'))
    if flag!=1:
        print("**********Bye****************")
        break
connec.close()


to continie enter 1:2
**********Bye****************


In [8]:
from IPython.display import clear_output
print("hello world")
clear_output(wait=True)
print("how are U")

how are U
